# scikit-learn（sklearn）でロジスティック回帰を動かす
**scikit-learn（sklearn）はPythonの機械学習ライブラリ**です．公式：https://scikit-learn.org/stable/

sklearnを用いると，驚くべきことに，**ライブラリのインポート・データの読み込みを除けば10行未満で学習・予測**まで行うことができます．
Pythonなので遅いのではないか（自分でCで書いた方が速いのではないか）と思うかもしれませんが，sklearnではNumPyやSciPyを上手く用いて書かれたコードや，Cで書かれた大変質の良いコードが動いているため，sklearnはかなり高速に動作します．
ほとんどの場合，雑に書いたCのプログラムよりは高速に動作します．
またPythonに慣れていなくとも，「慣れていないPythonでscikit-learnを使えるようになる時間」と「慣れているCで実装する時間」だと，おそらく圧倒的に前者の方が短いでしょう．

勿論，勉強のためにsklearnを使わずに実装しても構いませんが，本演習ではデータ分析全体の流れを体験してもらいたいため，サンプルコードではsklearnを用います．

## 今回学ぶこと
- pandasによるcsvの読み込みと簡単な操作（列・行へのアクセス，`numpy.ndarray`への変換）
- sklearnの基本的な使い方
- 提出までの流れ：
  1. 特徴ベクトルの作成
  2. モデル・アルゴリズムの選定，学習
  3. 予測
  4. 提出

## 予測モデル構築の流れ

本演習のように，機械学習を用いて予測モデルを構築し，未知の（テスト）データに対して予測を行う手順は，主に以下のようになります．

1. データを用意し，特徴ベクトルを作る
2. どのような手法（モデル）を使うかを決める
3. モデルを学習する方法を決め，学習する
4. 未知のデータに予測を行う（そして本演習では提出する）


## データ読み込み
演習で用いるデータはcsvファイル（comma-separated valuesファイル，つまりデータが`,`で区切られたファイル）で表現されています．
Pythonの標準の機能を使って読み込んでも良いのですが，Pythonには**pandasというデータ解析のための便利なライブラリ**があります．
pandasによって，CSVを含む様々なファイルを簡単に読み込み，さらに読み込んだデータに対して様々な処理を簡単に行うことができます．

なお，今回は文字列の情報を含んでいるためpandasを用いますが，数値情報だけであれば，Pythonにおける**数値・行列・線形代数計算ライブラリのNumPy**で読み込んで処理するほうが楽であることも多いです（NumPyについては，day1_2_how_to_numpy.ipynbを参照）．また，scikit-learnの関数には，NumPyの配列を前提とするものやNumPyの配列を返すものも数多くあります．そのため，今回の演習ではpandasで最低限の処理をした後，NumPyの配列に変換し（=いろいろな演算がサポートされている便利な配列），変換したものをscikit-learnに渡すことにします．
つまり，以下のような手順を行います：
1. `pandas`でデータを読み込む
2. `pandas`の便利な関数を使って，`numpy`の`ndarray`（`array`）に変換
3. （場合によっては）`numpy`の便利な関数を使って，特徴ベクトルを作成
4. scikit-learnに渡して学習

まず，今回使うものを`import`（=C言語で言うところのincludeだと思ってください）をしましょう．

In [ ]:
import numpy as np
import pandas as pd 
# 機械学習のライブラリ．今回はロジスティック回帰LogisticRegressionを動かす．
# LogisticRegressionはsklearnのlinear_modelモジュールの中にあるので，次のようにインポートする
from sklearn.linear_model import LogisticRegression

上のセルを動かしましたか？その際に特にエラーが出なければ，インポートに成功しています．
これ以降皆さんはnumpy，pandas，そしてsklearn.linear_modelの`LogisticRegression`が使えるようになりました！


では，次は訓練データとテストデータを読み込みます．
csvファイルは，pandasの`read_csv`という関数を使って読み込むことができます．
ファイルのパスを引数として渡すと，`pandas.DataFrame`という型で（オブジェクト指向言語なので，「`pandas.DataFrame`クラスのインスタンスを作成して」とも言える）読み込んだものを返してくれます．
詳細は[公式のドキュメント](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)を参照してください．
では実際に動かしてみます．

このノートブックを動かしているディレクトリにdataディレクトリがあり，その中にtrain.csvとtest.csvが存在していると仮定しています．
そうでない場合は適宜書き換えてください．

Goole Colabを用いている場合は，Google Driveにdataフォルダを置いて，次のセルを実行してください（自身のPython環境で動かしている場合は，次のセルは飛ばして良いです）．"このノートブックにGoogleドライブのファイルへのアクセスを許可しますか？"と表示された場合，「Googleドライブに接続」を選択し，リクエストを許可してください．

<details>
<summary>"Enter your authorization code" と表示された場合(ここをクリックすると展開します)</summary>

1. セルの実行時にURLが出てくるのでそこにアクセスしてください．
2. Googleアカウントのログインが求められるので，ログインしてください．
3. authorization codeが表示されるので，それを貼り付けてください．
</details>

In [ ]:
# Google Colab を用いている場合，このセルを実行
from google.colab import drive
drive.mount('/content/drive') # Google Driveをマウント（＝Colabから使えるようにする）

Google Driveがマウントできました．これで，Google Driveのデータを読み込むことができるようになります．

さっそく，dataフォルダからデータを取得してみましょう．

画面左側のバーで「フォルダ」のタブを選ぶと"drive"というフォルダが出てくると思いますが，そこがGoogle Driveを表しています．そこを見て`read_csv`で指定するパスを適宜書き換えてください．

In [ ]:
# Google Colab を用いている場合，このセルを実行
d_train = pd.read_csv("drive/My Drive/data/train.csv") # 訓練データを読み込む．TFがGoogle Driveの一番上にdataディレクトリを置いた場合はこのようなパスになった
d_test = pd.read_csv("drive/My Drive/data/test.csv") # テストデータを読み込む．TFがGoogle Driveの一番上にdataディレクトリを置いた場合はこのようなパスになった

In [ ]:
# 自身のPython環境で動かしている場合，このセルを実行（パスは適宜変更）
#d_train = pd.read_csv("data/train.csv") # 訓練データを読み込む
#d_test = pd.read_csv("data/test.csv") # テストデータを読み込む

エラーが何も出ていなければ読み込めています．しかし言われるがまま読み込んだだけでどうなっているのかよくわかりませんね．`print`してみましょう．

In [ ]:
print("訓練データ")
print(d_train)
print("\nテストデータ")
print(d_test)

train.csvの中身が出てきました！すばらしいですね．
詳細はスライドを確認してほしいですが，この行列は，各行が一人のデータに対応しています．
各列はすべての人に関するある1種類の情報を表していて，各列がどのような情報なのかはヘッダーに書かれています．
たとえば，2列目は"gender"と書かれているので，これは性別の情報ですね．
訓練データの最後の列は"stroke"と書かれており，これはまさしく目標値（出力・教師情報）です．
テストデータに"stroke"の列はなく，この列の値を予測して，その予測の正確さで皆さんに競っていただくのがこのコンペティションです．


さて，予測モデルを動かす前に，ちょっと`d_train`で遊んでみます．各列や行を取り出してみましょう．

列を取り出すのは非常に簡単で，他のプログラミング言語で配列の要素にアクセスするのと同じようにブラケット（角括弧）を用います．具体的には，

d_train[列の名前]

とすることで取り出せます．また，複数の列を取り出すことは，

d_train[列の名前のリスト]

でできます（"取り出す"と言っていますが，元の`DataFrame`から消えるわけではありません）．以下のセルで実際にやってみましょう．

In [ ]:
print("genderの列を取り出す")
print(d_train["gender"])

print("\nageとbmiの列を取り出す")
print(d_train[["age", "bmi"]])

print("\n取り出した結果を別の変数に格納することもできる")
d_train_date = d_train["work_type"]
print(d_train_date)


とりだせました！すばらしいですね．

次に行を取り出してみましょう．行も列と同様にブラケット（角括弧）で取り出せます．d_train\[i:j\]とすることで，$i$ 行目から $j-1$ 行目の値（つまり、$[i, j)$行目）を取り出すことができます．

In [ ]:
print("0から3(=4-1)行目まで取り出す")
print(d_train[0:4])
print("\n10行目だけ取り出す")
print(d_train[10:11])

できました．素晴らしいですね！他にも色々できますし，同じことを行う別の方法があったりもしますが，今回はとりあえずこの程度にしておきましょう．
詳細は例えば[pandasのデータ選択のドキュメント](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html)
等を見てみてください．

さて，次に特徴ベクトルを作っていきたいのですが，その前にいくつかの変数を準備します．

まず，訓練データとテストデータの数を表す変数`n_train`と`n_test`を準備します．今日は使いませんが，今後よく使うためここで取得の仕方を確認しておきます．
各行が一つのデータに対応しているので，**`d_train`や`d_test`の行の数**が取得できればよいのですが，これは`len`関数によって取得できます．

また，sklearnでモデルを学習させる際に必要となるので，訓練データの目標値"stroke"を`d_train`から取り出して分離させておきます．
`pop`メソッドを使うことで，`DataFrame`（`d_train`や`d_test`）から特定の列を分離させることができます．
「分離」と言っている通り，ブラケットによるアクセスと違い，`pop`で取り出すと**元の`DataFrame`からその列は削除されます**．

In [ ]:
print("訓練データとテストデータの数を取得")
n_train = len(d_train)
n_test = len(d_test)
print(f"訓練データ数：{n_train}，テストデータ数：{n_test}")
print("\n 出力情報を取り出す．strokeの列が消えている")
# targetの値
y_train = d_train.pop('stroke')
print(d_train)
print(y_train)
print(f"y_trainのクラスは{type(y_train)}")

訓練データ数は`3577`，テストデータ数は`1533`で，これはスライドやコンペサイトに書いてあるとおりです．
また，`d_train`から"stroke"の列が消えていて，`y_train`が"stroke"の情報を持っていますね．

今，`y_train`は怪しい（？）クラスのインスタンスになっています．別にこのままでもよいのですが，後々のことを考えると，NumPyの配列の方が都合が良いです．
便利なことに，`y_train.values`とすることで，列の名前や行の番号が消え，実際に持っている値だけをNumPyの配列として取り出すことができます．
`y_train`は`Series`クラスのインスタンスになっていますが，これは`DataFrame`クラスでも同じです．
つまり，`d_train.values`とすることで，同じく行番号や列の名前を消して，中の値だけをNumPyの配列として取り出すことができます．

In [ ]:
y_train = y_train.values
print(y_train)
print(f"y_trainのクラスは{type(y_train)}")

### ここまでのまとめ
- `pandas`の`read_csv`でcsvファイルを読み込むことができる．読み込んだファイルは`DataFrame`クラスのインスタンスとして返される．
- `DataFrame`クラスのインスタンスは`print`で中身を簡単にわかりやすく見ることができる．
- `DataFrame`では，[列の名前]や[列のリスト]とすることで，特定の列を取り出すことができる．
- `DataFrame`では，[i:j]とすることで，特定の範囲の行を取り出すことができる．
- `.values`で`DataFrame`や`DataFrame`から取り出した列の持つ値を，NumPyの配列`np.array`として取り出すことができる．

## 特徴ベクトルを作る

さて，いよいよ特徴ベクトルを作っていきましょう．入力の情報として，以下が与えられています：

 - id：整数値．1人ごとに割り当てられる固有の数値．
 - gender：文字列．性別を意味し，値は"Male"，"Female"，"Other"のいずれか．
 - age：整数値．年齢．
 - hypertension：整数値．高血圧患者なら1，でなければ0．
 - heart_disease：整数値．心臓病患者なら1，でなければ0．
 - ever_married：文字列．結婚経験があるかを意味し，値は"No"か"Yes"のいずれか．
 - work_type：文字列．労働形態を意味し，値は"children"，"Govt_jov"，"Never_worked"，"Private"，"Self-employed"のいずれか．
 - Residence_type：文字列．居住地のタイプを意味し，値は"Rural"か"Urban"のいずれか．
 - avg_glucose_level：実数値．平均血糖値．
 - bmi：実数値．ボディマス指数(Body Mass Index)．
 - smoking_status：文字列．喫煙習慣を意味し，値は"formerly smoked"，"never smoked"，"smokes"，"Unknown"のいずれか．
 - stroke：**今回予測する値**．整数値．脳卒中患者なら1，でなければ0．

これらの情報を見てみると，数値の情報だけでなく，文字列の情報も含まれていることが分かります．しかし，機械学習のアルゴリズムの多くは各データが「数値のベクトル」として表されていると仮定しており，このような**文字列の情報はそのまま扱えません**．
今回動かしてみるロジスティック回帰も，数値の情報しか扱えません．
機械学習や統計では，（基本的には）**一つ一つのデータは数値のベクトル**として表現され，**データの集まりは数値の行列**として表現されている，と考えます（一つ一つの行が一つのデータに対応し，一つ一つの列が何らかの情報（特徴）に対応）．同様に，**訓練データの目標値（ここではstroke）の集まりは数値のベクトル**として表現されていると考えます．
sklearnを使って学習する際も，訓練データの入力を表す行列と，訓練データの目標値を表すベクトルを渡す必要があります．

ここまで皆さんは**ただ読んで動かしてきただけで退屈**だったかと思います．**そこでQuizです**．

### Quiz

以下のセルを完成させて，`d_train`と`d_test`から"id"，"age"，"hypertension"，"heart_disease"，"avg_glucose_level"の5つの列を取り出して作った，`X_train_num`と`X_test_num`を作成しなさい．ここで，`X_train_num`と`X_test_num`はどちらもNumPyの配列`np.ndarray`（`np.array`）であるとします．
なお，`columns_num`は負担を削減するためこちらで用意したリストです．使わなくても良いですが，使ったほうが楽かと思います．

**Quizの解答はこのノートブックの最下部にあります．**

In [ ]:
columns_num = ["id","age","hypertension","heart_disease","avg_glucose_level"]
X_train_num = 
X_test_num = 

できたでしょうか？中身を出力してみます．上の`d_train`や`d_test`と見比べてみてください．
`type`や`shape`の`print`では，

 - 訓練データ：<class 'numpy.ndarray'> (3577, 5)
 - テストデータ：<class 'numpy.ndarray'> (1533, 5)
 
と表示されていればよいです（`.shape`は配列の形を表しています）．

In [ ]:
print("X_train_numを表示")
print(type(X_train_num), X_train_num.shape)
print(X_train_num)
print("\nX_test_numを表示")
print(type(X_test_num), X_test_num.shape)
print(X_test_num)

なお，この配列はそれぞれ列の数が5つ，つまり各データが5つの数値的な情報によって表現されている，ということになります．
このようなデータを表す情報のことを **特徴（feature）** といい，特徴のベクトルのことをそのまま **特徴ベクトル** ，特徴の数を
**（データの，あるいは特徴の）次元**と言います．つまり，今は5次元の特徴ベクトルとしてデータを表現している，といえます．

## ロジスティック回帰を動かす
ここまでおつかれさまでした．**ついに機械学習アルゴリズムを動かす**ときが来ました．
sklearnの教師あり学習のモデル・アルゴリズムの実装について，以下に列挙されています：

https://scikit-learn.org/stable/supervised_learning.html

凄まじい量ですね．色々ありますが，解く問題が分類問題か回帰問題かによって使用する（のに適した）モデルは変わります（その他にも様々な問題がありますが，ここでは割愛します）．今回取り組む問題は（2値）分類問題なので，（2値）分類問題に対する単純な手法の一つである**ロジスティック回帰 = LogisticRegression**を動かしてみます．ロジスティック「回帰」という名前を見て疑問に思われる方もいるかもしれませんが，ロジスティック回帰モデルは回帰よりむしろ分類のためのモデルとして扱われています．

まず，ロジスティック回帰について簡単に説明しておきます．
ロジスティック回帰モデルは，以下のように定式化されます．

$$
 y(\mathbf{x}; \mathbf{w}) = \sigma \left(\mathbf{w}^{\top} \mathbf{x} \right) = \frac{1}{1+\exp(-\sum_{i=1}^{D}w_ix_i)}
$$

ここで，$\mathbf{x} = (x_1,\ldots,x_D)^{\top} \in \mathbb{R}^{D}$は特徴ベクトル，$D$はその次元（今回の例だと$D=5$），$\mathbf{w} = (w_1,\ldots,w_D)^{\top} \in \mathbb{R}^{D}$は学習されるパラメータです（$x_1$や$x_2$というのが一つの特徴を表しています）．また，$\sigma$はシグモイド関数$\sigma(a) = 1/(1+\exp(-a)) (a \in \mathbb{R})$です．このとき$\sigma(a) \in [0,1]$であり，上で定式化された $y$ はとある確率，すなわち「入力 $x$ がとあるクラスに属する確率」として捉えることができます．なお，1変数に対するシグモイド関数は以下のような形となります．

In [ ]:
import matplotlib.pyplot as plt
import math

a = np.arange(-10, 10, 0.1)

y = 1 / (1 + math.e**-a)

plt.plot(a, y)
plt.xlabel('a')
plt.ylabel(r'$\sigma(a)$')
plt.show()

また，学習の際には一般に指標となる目的関数（誤差関数）を設定し，この関数を最小化することで学習を進めます．ロジスティック回帰での学習において，今回は目的関数として交差エントロピー誤差関数（cross-entropy error function）を使うこととします．交差エントロピー誤差関数は，以下のように定義されます．

$$
  E(\mathbf{w}) = - \sum_{n=1}^N \{ t_n \ln y_n + (1-t_n) \ln(1-y_n) \} 
$$

ただし，$y_n = \sigma ( \mathbf{w}^{\top} {\mathbf{x}_n} ) $($n$番目のデータに対する$y$の予測値)，$t_n \in \{0,1\}$($n$番目のデータの正解クラス)です．

ここまで今回扱うロジスティック回帰モデルについて見てきましたが，習うより慣れろ，ということで早速予測をしてみましょう．

sklearnを用いる基本的な手順は，
1. モデルのインスタンスを作成※
2. 作成したモデルオブジェクトを **`fit`メソッド** を用いて学習．`fit`メソッドには訓練データの入力と目標値（つまり，行列とベクトル）を渡す．
3. 学習したモデルを用いて **`predict`メソッド** で予測．`predict`メソッドにはデータの入力（つまり行列）を渡す．

となっています．

※sklearnでは学習アルゴリズムが違う場合は別のクラスとして提供されていることが多いです．
また，一つのクラスの中で学習アルゴリズムを選べる場合も，インスタンスを作成する時に学習アルゴリズムを指定します．
したがって，sklearnでは「モデルのインスタンスを作成」した時点で学習アルゴリズムを選んだことになります．

では実際に上の手順通りに学習してみます．
Pythonにおいて，`ClassA`という名前のクラスのインスタンスは`ClassA()`で作ることができるのでした（day1_1_how_to_python.ipynb参照）．
したがって，今回のケースでは以下のようになります．

In [ ]:
# 手順1：LogisticRegressionのインスタンスの作成
lr = LogisticRegression()
# 手順2：上で作ったオブジェクトの学習
lr.fit(X_train_num, y_train)
# 手順3：テストデータに対する予測
y_pred_test_lr = lr.predict(X_test_num)

**これで学習とテストデータに対する予測まで終わりました!**
たった3行です．すごいですね．予測結果をプリントしてみましょう．

In [ ]:
print(y_pred_test_lr)

予測の正しさに関してはひとまずおいておき，この予測を提出するとどのような結果になるか見てみましょう．

## 予測結果の提出

予測ができたので，この結果を提出することを考えます．予測結果をファイルに出力しなければなりません．ファイル出力ときくと面倒そうだと思うかもしれませんが，NumPyを用いれば以下のように簡単にできます．
`np.savetxt`の`X`という引数で保存したい配列を指定し，`fname`という引数でファイル名を指定します．
今回は，`y_pred_test_lr`を`y_pred_lr.txt`という名前で保存することにします．

In [ ]:
np.savetxt(X=y_pred_test_lr, fname='y_pred_lr.txt')

y_pred_lr.txtというファイルが生成されました（Google Colabの場合は，画面左側のタブが「ファイル」のであるとき，「ファイル」という文字の下に3つアイコンがありますが，その真ん中の「更新」アイコンを押すと出てくるはずです）．これを提出してみましょう（Google Colabの場合はまず生成されたy_pred_lr.txtをダウンロードしてください）．正しく提出できていれば，予測に対してスコアが表示されるはずです．

私の手元環境では，「予測結果が提出されました．今回のスコアは0.500000です．」と表示されました．

このスコアは，果たして良いスコアなのでしょうか？これを調べるために，まず今回のコンペで使用している「AUC」という評価指標について見ていきましょう．

### ROC曲線（Receiver Operating Characteristic Curve）について

ROC曲線とは縦軸に真陽性率（TPR, True Positive Rate, 陽性であるデータのうち，正しく陽性と予測したデータの割合），横軸に偽陽性率（FPR, False Positive Rate, 陰性であるデータのうち，間違えて陽性と予測したデータの割合）をとり，モデルの閾値を変化させたときの真陽性率と偽陽性率の変化をプロットして得られる曲線のことです．今回は，正解ラベルが1であるデータを陽性，0であるデータを陰性とし，予測値が閾値以上であれば陽性，でなければ陰性と予測するものとします．
例えば，データの正解ラベルが

```Python
y = [0,1,0,1]
```

であって，ラベルの予測が

```Python
y_pred = [0.2,0.5,0.4,0.3]
```

であったとします（このとき予測値は実数値であり，予測の確信度合いとみることができます）．
ここで閾値を変化させて真陽性率と偽陽性率を調べてみると，

|  閾値  |  真陽性率  |  偽陽性率  |
| :---: | :---: |  :---:  |
|  0.15  |  1.0  |  1.0  |
|  0.25  |  1.0  |  0.5  |
|  0.35  |  0.5  |  0.5  |
|  0.45  |  0.5  |  0.0  |
|  0.55  |  0.0  |  0.0  |

となり，これをグラフにプロットすると以下のようになります．

In [ ]:
TPR = [1.0,1.0,0.5,0.5,0.0]
FPR = [1.0,0.5,0.5,0.0,0.0]

plt.fill_between(FPR,TPR,0,color='g')
plt.plot(FPR,TPR,c='r')
plt.scatter(x=FPR,y=TPR,c='k')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.show()

閾値の変化に合わせてプロットした点（図中の黒い点）を線で結ぶと，図の左下から右上に向かって1本の曲線（図中の赤い線）が引かれます．この曲線をROC曲線といいます．
AUCは，このROC曲線の下側（図中の緑色の部分）の面積として定義されます．上の図の場合，AUCは0.75となります．

AUCの主な性質としては，
- データを完全に分類できたとき，AUCは1になる
- データを完全にランダムに分類したとき，AUCはおよそ0.5になる

というものがあります．つまり，先ほど得られた0.5というスコアは，完全にランダムに予測するモデルとほぼ同じスコアということになってしまいます．

実は，先ほどの予測ではすべてのデータに対して予測値が0となっており，一般にこのときのAUCはちょうど0.5になります．
AUCの算出方法を考えると，予測値は整数値ではなく，実数値として各データの予測に確信度合いの強弱を持たせたほうが良さそうに思えます．
そこで，今後は予測値を実数値で出すことを考えます．

実は，sklearnのロジスティック回帰モデルには`predict_proba()`というメソッドがあり，予測ラベルが1である確率を出力することができます．早速，試してみましょう．

In [ ]:
y_pred_test_lr_proba = lr.predict_proba(X_test_num)
print(y_pred_test_lr_proba)
# i番目の要素は[予測ラベルが0の確率, 予測ラベルが1の確率]
# 例：[0.98493579 0.01506421] → 約98.5%の確率で予測ラベルは0と予測される

実数値での出力を出すことができました！提出の際は1次元で出力する必要があるので，「各データに対する予測ラベルが1の確率」を出力することにします．

In [ ]:
np.savetxt(X=y_pred_test_lr_proba[:,1], fname='y_pred_lr_proba.txt') #予測ラベルが1の確率を出力

今日の資料はここで終了です．おつかれさまでした．今日の資料では提出までの流れを追っていきましたが，次回以降はスコアの改善に向けて他のモデルを実装するなど，様々な取り組みを行っていきます．
興味のある人はどんどん進んでみるのもいいでしょう．
また，今日作成したロジスティック回帰モデルについて詳しく知りたい人は
[sklearnのリファレンス](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.predict_proba)等を読んでみるとよいと思います．

(余裕のある人は`y_pred_lr_proba.txt`を提出してみて，そのスコアを見てみるのも良いでしょう．何か気づくことがあるかもしれません．)

## まとめその2
 - sklearnでは以下の手順で予測モデルを作成，学習しテストデータの予測を行う：(1)インスタンスを作成し，(2)`fit`メソッドでモデルを学習，(3)`predict`メソッドでテストデータの目標値を予測．
 - `np.savetxt`で予測結果をファイルに書き込むことができる．
 - 多くの予測モデルにはユーザが決めるパラメータがあり，それらをハイパーパラメータという．ハイパーパラメータの値によって結果は大きく変わることもあるため，試行錯誤の余地がある．
 - 「文字列の特徴をどうするか」「提出をする前の定量的な評価」については次回行う ．

## Answer

`DataFrame`に対して，ブラケット（角括弧）[]を使うことで特定の列を取り出せるのでした．また，`.values`で`DataFrame`の値のみを持つ`np.array`を作成できるのでした．

In [ ]:
columns_num = ["id","age","hypertension","heart_disease","avg_glucose_level"]
X_train_num = d_train[columns_num].values
X_test_num = d_test[columns_num].values